In [61]:
import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt

import os
import datetime

In [62]:
# image size
MODEL_INPUT_IMAGE_SIZE = [200, 200]
# Fraction of the dataset to be used for testing. 70% for training 30% for testing
TRAIN_TEST_SPLIT = 0.3
# Number of samples to take from dataset
N = 20000

In [63]:
def parse_image(filename):

    # Read the image from the filename and resize it.
    print(filename)
    image_raw = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_raw, channels=3)
    image = tf.image.resize(image, MODEL_INPUT_IMAGE_SIZE) / 255

    # Split the filename to get the age and the gender. Convert the age ( str ) and the gender ( str ) to dtype float32.
    parts = tf.strings.split(tf.strings.split(filename, '\\')[-1], '_')

    # Normalize
    # age = tf.strings.to_number(parts[0]) // 10
    age = tf.strings.to_number( parts[ 0 ] ) / 116
    print(age)
    return image, age


# List all the image files in the given directory.
list_ds = tf.data.Dataset.list_files('./UTKFace/*.jpg', shuffle=True)

# Map `parse_image` method to all filenames.
dataset = list_ds.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.take(N)

Tensor("args_0:0", shape=(), dtype=string)
Tensor("truediv_1:0", shape=(), dtype=float32)


In [64]:
# Create train and test splits of the dataset.
num_examples_in_test_ds = int(dataset.cardinality().numpy() * TRAIN_TEST_SPLIT)

test_ds = dataset.take(num_examples_in_test_ds)
train_ds = dataset.skip(num_examples_in_test_ds)

print('Num examples in train ds {}'.format(train_ds.cardinality()))
print('Num examples in test ds {}'.format(test_ds.cardinality()))

Num examples in train ds 14000
Num examples in test ds 6000


In [65]:
# # Negative slope coefficient for LeakyReLU.
# leaky_relu_alpha = 0.2

# lite_model = False

# output_size = 12

# # Define the conv block.
# def conv(x, num_filters, kernel_size=(3, 3), strides=1):
#     if lite_model:
#         x = tf.keras.layers.SeparableConv2D(num_filters,
#                                             kernel_size=kernel_size,
#                                             strides=strides, 
#                                             use_bias=False,
#                                             kernel_initializer=tf.keras.initializers.HeNormal(),
#                                             kernel_regularizer=tf.keras.regularizers.L2(1e-5)
#                                             )(x)
#     else:
#         x = tf.keras.layers.Conv2D(num_filters ,
#                                    kernel_size=kernel_size ,
#                                    strides=strides,
#                                    use_bias=False,
#                                    kernel_initializer=tf.keras.initializers.HeNormal() ,
#                                    kernel_regularizer=tf.keras.regularizers.L2(1e-5)
#                                    )(x)

#     x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.LeakyReLU(leaky_relu_alpha)(x)
#     return x

# def dense(x, filters, dropout_rate):
#     x = tf.keras.layers.Dense(filters, kernel_regularizer=tf.keras.regularizers.L2(0.1), bias_regularizer=tf.keras.regularizers.L2(0.1))(x)
#     x = tf.keras.layers.LeakyReLU(alpha=leaky_relu_alpha)(x)
#     x = tf.keras.layers.Dropout(dropout_rate)(x)
#     return x


# # No. of convolution layers to be added.
# num_blocks = 6
# # Num filters for each conv layer.
# num_filters = [16, 32, 64, 128, 256, 256]
# # # Kernel sizes for each conv layer.
# # kernel_sizes = [3, 3, 3, 3, 3, 3]

# # Init a Input Layer.
# inputs = tf.keras.layers.Input(shape=MODEL_INPUT_IMAGE_SIZE + [3])

# # Add conv blocks sequentially
# x = inputs
# for i in range(num_blocks):
#     # x = conv(x, num_filters=num_filters[i], kernel_size=kernel_sizes[i])
#     x = conv(x, num_filters=num_filters[i])
#     x = tf.keras.layers.MaxPooling2D()(x)

# # Flatten the output of the last Conv layer.
# x = tf.keras.layers.Flatten()(x)
# conv_output = x 

# # Add Dense layers ( Dense -> LeakyReLU -> Dropout )
# x = dense(conv_output, 256, 0.6)
# x = dense(x, 64, 0.4)
# x = dense(x, 32, 0.2)
# outputs = tf.keras.layers.Dense(output_size , activation='relu')(x)

# # Build the Model
# model = tf.keras.models.Model(inputs , outputs)

# # Uncomment the below to view the summary of the model.
# model.summary()
# # tf.keras.utils.plot_model( model , to_file='architecture.png' )

In [66]:
leaky_relu_alpha = 0.2
lite_model = False
output_size = 1

def conv2D(x, num_filters, kernel_size=(3, 3), strides=1):
    x = tf.keras.layers.Conv2D(num_filters ,
                                kernel_size=kernel_size ,
                                strides=strides,
                                use_bias=False,
                                kernel_initializer=tf.keras.initializers.HeNormal() ,
                                kernel_regularizer=tf.keras.regularizers.L2(1e-5))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.LeakyReLU(leaky_relu_alpha)(x)
    return x

def dense(x, filters, dropout_rate):
    x = tf.keras.layers.Dense(filters, kernel_regularizer=tf.keras.regularizers.L2(0.1), bias_regularizer=tf.keras.regularizers.L2(0.1))(x)
    x = tf.keras.layers.LeakyReLU(alpha=leaky_relu_alpha)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    return x

cov_filters = [32, 64, 128, 256, 256]
dense_filters = [128, 64, 32]
dropout_filters = [0.3, 0.3, 0.2]


inputs = tf.keras.layers.Input(shape=MODEL_INPUT_IMAGE_SIZE + [3])

# Add conv blocks sequentially
x = inputs
# print(len(cov_filters))
for i in range(len(cov_filters)):
    x = conv2D(x, num_filters=cov_filters[i])
    x = tf.keras.layers.MaxPooling2D()(x)

# Flatten the output of the last Conv layer.
x = tf.keras.layers.Flatten()(x)
# conv_output = x 

# Add Dense layers ( Dense -> LeakyReLU -> Dropout )
for i in range(len(dense_filters)):
    x = dense(x, dense_filters[i], dropout_filters[i])
    
outputs = tf.keras.layers.Dense(output_size , activation='relu')(x)

# Build the Model
model = tf.keras.models.Model(inputs , outputs)

# Uncomment the below to view the summary of the model.
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 200, 200, 3)]     0         
                                                                 
 conv2d_80 (Conv2D)          (None, 198, 198, 32)      864       
                                                                 
 batch_normalization_78 (Bat  (None, 198, 198, 32)     128       
 chNormalization)                                                
                                                                 
 leaky_re_lu_102 (LeakyReLU)  (None, 198, 198, 32)     0         
                                                                 
 max_pooling2d_78 (MaxPoolin  (None, 99, 99, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_81 (Conv2D)          (None, 97, 97, 64)        1843

In [67]:
# Initial learning rate
learning_rate = 0.001

num_epochs = 50
batch_size = 128

patience = 10

# Batch and repeat `train_ds` and `test_ds`.
train_ds = train_ds.batch(batch_size)
test_ds = test_ds.batch(batch_size)

# Init ModelCheckpoint callback
save_dir_ = 'model_1'
save_dir = save_dir_ + '/{epoch:02d}-{val_mae:.2f}.h5'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( 
    save_dir,
    save_best_only=True,
    monitor='val_mae',
    mode='min'
)

# Init LR Scheduler
def scheduler( epochs , learning_rate ):
    if epochs < num_epochs * 0.25:
        return learning_rate
    elif epochs < num_epochs * 0.5:
        return 0.0005
    elif epochs < num_epochs * 0.75:
        return 0.0001
    else:
        return 0.000095

lr_schedule_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

# Init Early Stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_mae', patience=patience)

# Compile the model
model.compile( 
    loss=tf.keras.losses.mean_absolute_error,
    optimizer = tf.keras.optimizers.Adam(learning_rate), 
    metrics=['mae', 'sparse_categorical_accuracy']
)

In [68]:
model.fit( 
    train_ds,
    epochs=num_epochs,
    validation_data=test_ds,
    callbacks=[checkpoint_callback, lr_schedule_callback, early_stopping_callback]
)

Epoch 1/50


ResourceExhaustedError:  OOM when allocating tensor with shape[128,64,97,97] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_8/batch_normalization_79/FusedBatchNormV3
 (defined at d:\Python\Python37\lib\site-packages\keras\layers\normalization\batch_normalization.py:599)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_64718]

Errors may have originated from an input operation.
Input Source operations connected to node model_8/batch_normalization_79/FusedBatchNormV3:
In[0] model_8/conv2d_81/Conv2D (defined at d:\Python\Python37\lib\site-packages\keras\layers\convolutional.py:238)	
In[1] model_8/batch_normalization_79/ReadVariableOp:	
In[2] model_8/batch_normalization_79/ReadVariableOp_1:	
In[3] model_8/batch_normalization_79/FusedBatchNormV3/ReadVar

In [ ]:
model.save('model0606-1.h5')

In [ ]:
p = model.evaluate(test_ds)
print(p)

47/47 [==============================] - 87s 2s/step - loss: 0.3883 - mae: 0.2627 - sparse_categorical_accuracy: 0.1357
[0.388266921043396, 0.2627376317977905, 0.13566666841506958]
